# Starter Tutorial

This tutorial provides step-by-step instructions for getting started using EDSL (*Expected Parrot Domain-Specific Language*), an open-source Python library for simulating surveys, experiments and other research tasks using AI agents and large language models.
EDSL is developed by [Expected Parrot](https://www.expectedparrot.com/about) and available under the MIT License.
The source code is hosted on [GitHub](https://github.com/expectedparrot/edsl). 

## Goals of this tutorial
We begin with technical setup: instructions for installing the EDSL library and storing API keys to access language models.
Then we demonstrate some of the basic features of EDSL, with examples for constructing and running surveys with agents and models, and analyzing responses as datasets.
By the end of this tutorial, you will be able to use EDSL to do each of the following:

* Construct various types of questions tailored to your research objectives.
* Combine questions into surveys and integrate logical rules to control the survey flow.
* Design personas for AI agents to simulate responses to your surveys.
* Choose and deploy large language models to generate responses for AI agents.
* Analyze responses as datasets with built-in analytical tools.

## Storing & sharing your work
We also introduce [Coop](https://www.expectedparrot.com/content/explore): a platform for creating, storing and sharing AI-based research.
Coop is fully integrated with EDSL and free to use. 
At the end of the tutorial we show how to use EDSL with Coop by posting content created in this tutorial for anyone to view at the web app.

## Further reading & questions
Please see our [documentation page](https://docs.expectedparrot.com) for more details on each of the topics covered in this notebook.
If you encounter any issues or have questions, please email us at info@expectedparrot.com or post a question at our [Discord channel](https://discord.com/invite/mxAYkjfy9m).

## Pre-requisites
EDSL is compatible with Python 3.9 - 3.12.
Before starting this tutorial, please ensure that you have a Python environment set up on your machine or in a cloud-based environment, such as Google Colab.
You can find instructions for installing Python at the [Python Software Foundation](https://www.python.org/downloads/).

## Recommendations 
The code examples in this tutorial are designed to be run in a Jupyter notebook or another Python environment, or in a cloud-based environment such as Google Colab.

If you are using Google Colab, please see additional instructions for setting up EDSL in the [Colab setup](https://docs.expectedparrot.com/en/latest/colab_setup.html) page in the documentation.

We also recommend using a virtual environment when installing and using EDSL in order to avoid conflicts with other Python packages.
You can find instructions for setting up a virtual environment at the [Python Packaging Authority](https://packaging.python.org/guides/installing-using-pip-and-virtual-environments/).

## Installation
To begin using EDSL, you first need to install the library. 
This can either be done locally on your machine or in a cloud-based environment, such as Google Colab.
Once you have decided where to install EDSL, you can choose to whether install it from [PyPI](https://pypi.org/project/edsl/) or [GitHub](https://github.com/expectedparrot/edsl):

### From PyPI
Install EDSL directly using `pip`, which is straightforward and recommended for most users. 
We also recommend using a virtual environment to manage your Python packages (see *Recommendations* above).
Uncomment and run the following command to install EDSL from PyPI:

In [1]:
# pip install edsl

If you have already installed EDSL, you can uncomment and run the following code to check that your version is up to date (compare it to the version at [PyPI](https://pypi.org/project/edsl/)):

In [2]:
# pip show edsl

If your version of EDSL is not up to date, uncomment and run the following code to update it:

In [3]:
# pip install --upgrade edsl

### From GitHub
You can find the source code for EDSL and contribute to the project at [GitHub](https://github.com/expectedparrot/edsl).
Installing from GitHub allows you to get the latest updates to EDSL before they are released to a new version at PyPI.
This is recommended if you are using new features or contributing to the project.
Uncomment and run the following command to install EDSL from GitHub:

In [4]:
# pip install git+https://github.com/expectedparrot/edsl.git@main

## Accessing LLMs
The next step is to decide how you want to access language models for running surveys.
EDSL works with many popular language models that you can choose from to generate responses to your surveys.
These models are hosted by various service providers, such as Anthropic, Azure, Bedrock, Deep Infra, Google, Groq, Mistral, OpenAI, Replicate and Together.
In order to run a survey, you need to provide API keys for the service providers of models that you want to use.
There are two methods for providing API keys to EDSL:

* Use an Expected Parrot API key to access all available models
* Provide your own API keys from service providers

## Create an account
The easiest way to manage your keys is from your Expected Parrot account.
[Create an account](https://www.expectedparrot.com/login) with an email address and then navigate to your [Settings](https://www.expectedparrot.com/home/api) page to view your Expected Parrot API key. 
It is stored automatically and can be regenerated at any time.
You will also see options for activating remote inference and caching; this allows your surveys to be run and your results to be stored remotely at the Expected Parrot server instead of your own machine.

## Managing keys
If you want to use your own keys to run surveys, navigate to your [Keys](https://www.expectedparrot.com/home/keys) page and use the options to add keys and optionally share access to them with other users. 
You can specify which keys to use at any time, and check the current priority of your keys. 
Your Expected Parrot API key is used by default.

Please see [instructions](https://www.expectedparrot.com/getting-started/edsl-api-keys) for alternative methods of storing your own API keys.

*Note:* If you try to run a survey without storing a required API key, you will be provided a link to activate remote inference and use your Expected Parrot API key.

## Credits & tokens
Running surveys with language models requires tokens.
If you are using your own API keys, service providers will bill you directly.
If you are using your Expected Parrot API key to access models, you will need to purchase credits to cover token costs.
Please see the model pricing page for details on available models and their current prices.

*Note:* Your account comes with 100 free credits. You can purchase more credits at any time at your [Credits](https://www.expectedparrot.com/home/purchases) page.

After installing EDSL and storing API keys you are ready to run some examples!

## Example: Running a simple question
EDSL comes with a [variety of question types](https://docs.expectedparrot.com/en/latest/questions.html) that we can choose from based on the form of the response that we want to get back from a model.
To see a list of all question types:

In [5]:
from edsl import Question

Question.available()

,question_type,question_class,example_question
0,checkbox,QuestionCheckBox,"Question('checkbox', question_name = """"""never_eat"""""", question_text = """"""Which of the following foods would you eat if you had to?"""""", min_selections = 2, max_selections = 5, question_options = ['soggy meatpie', 'rare snails', 'mouldy bread', 'panda milk custard', 'McDonalds'], include_comment = False, use_code = True)"
1,dict,QuestionDict,"Question('dict', question_name = """"""example"""""", question_text = """"""Please provide a simple recipe for hot chocolate."""""", answer_keys = ['title', 'ingredients', 'num_ingredients', 'instructions'], value_types = ['str', 'list[str]', 'int', 'str'], value_descriptions = ['The title of the recipe.', 'A list of ingredients.', 'The number of ingredients.', 'The instructions for making the recipe.'], question_presentation = """"""Please provide a simple recipe for hot chocolate."""""", answering_instructions = """"""Please respond with a dictionary using the following keys: title, ingredients, num_ingredients, instructions. Here are descriptions of the values to provide: - ""title"": ""The title of the recipe."" - ""ingredients"": ""A list of ingredients."" - ""num_ingredients"": ""The number of ingredients."" - ""instructions"": ""The instructions for making the recipe."" The values should be formatted in the following types: - ""title"": ""str"" - ""ingredients"": ""list[str]"" - ""num_ingredients"": ""int"" - ""instructions"": ""str"" If you do not have a value for a given key, use ""null"". After the answer, you can put a comment explaining your response on the next line. """""")"
2,extract,QuestionExtract,"Question('extract', question_name = """"""extract_name"""""", question_text = """"""My name is Moby Dick. I have a PhD in astrology, but I'm actually a truck driver"""""", answer_template = {'name': 'John Doe', 'profession': 'Carpenter'})"
3,free_text,QuestionFreeText,"Question('free_text', question_name = """"""how_are_you"""""", question_text = """"""How are you?"""""")"
4,functional,QuestionFunctional,"Question('functional', question_name = """"""sum_and_multiply"""""", question_text = """"""Calculate the sum of the list and multiply it by the agent trait multiplier."""""")"
5,likert_five,QuestionLikertFive,"Question('likert_five', question_name = """"""happy_raining"""""", question_text = """"""I'm only happy when it rains."""""", question_options = ['Strongly disagree', 'Disagree', 'Neutral', 'Agree', 'Strongly agree'])"
6,linear_scale,QuestionLinearScale,"Question('linear_scale', question_name = """"""ice_cream"""""", question_text = """"""How much do you like ice cream?"""""", question_options = [1, 2, 3, 4, 5], option_labels = {1: 'I hate it', 5: 'I love it'})"
7,list,QuestionList,"Question('list', question_name = """"""list_of_foods"""""", question_text = """"""What are your favorite foods?"""""", max_list_items = None)"
8,matrix,QuestionMatrix,"Question('matrix', question_name = """"""child_happiness"""""", question_text = """"""How happy would you be with different numbers of children?"""""", question_items = ['No children', '1 child', '2 children', '3 or more children'], question_options = [1, 2, 3, 4, 5], option_labels = {1: 'Very sad', 3: 'Neutral', 5: 'Extremely happy'})"
9,multiple_choice,QuestionMultipleChoice,"Question('multiple_choice', question_name = """"""how_feeling"""""", question_text = """"""How are you?"""""", question_options = ['Good', 'Great', 'OK', 'Bad'], include_comment = False)"


We can see the components of a particular question type by importing the question type class and calling the `example` method on it:

In [6]:
from edsl import (
    # QuestionCheckBox,
    # QuestionExtract,
    # QuestionFreeText,
    # QuestionFunctional,
    # QuestionLikertFive,
    # QuestionLinearScale,
    # QuestionList,
    QuestionMultipleChoice,
    # QuestionNumerical,
    # QuestionRank,
    # QuestionTopK,
    # QuestionYesNo
)

q = QuestionMultipleChoice.example() # substitute any question type class name
q

,key,value
0,question_name,how_feeling
1,question_text,How are you?
2,question_options:0,Good
3,question_options:1,Great
4,question_options:2,OK
5,question_options:3,Bad
6,include_comment,False
7,question_type,multiple_choice


Here we create a simple multiple choice question of our own:

In [7]:
from edsl import QuestionMultipleChoice

q = QuestionMultipleChoice(
    question_name = "smallest_prime",
    question_text = "Which is the smallest prime number?",
    question_options = [0, 1, 2, 3]
)

We can administer the question to a language model by calling the `run` method on it. 
If you have activated remote inference and stored your Expected Parrot API key (see instructions above), the question will be run remotely at the Expected Parrot server.
Results are stored at an unlisted Coop page by default; we can also set the visibility to `public` or `private` either when we run it or by updating the object (demonstrated in later examples).
We can also view a progress report for the job:

In [8]:
results = q.run()

Job UUID,b2a05cba-de90-4988-8dcc-b5c542db37c5
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/b2a05cba-de90-4988-8dcc-b5c542db37c5
Exceptions Report URL,None
Results UUID,8392c145-d411-498a-8547-627eb5744c1a
Results URL,https://www.expectedparrot.com/content/8392c145-d411-498a-8547-627eb5744c1a


### Inspecting results
This generates a dataset of `Results` that we can readily access with [built-in methods for analysis](https://docs.expectedparrot.com/en/latest/results.html). 
Here we `select()` the response to inspect it, together with the model that was used and the model's "comment" about its response--a field that is automatically added to all question types other than free text:

In [9]:
results.select("model", "smallest_prime", "smallest_prime_comment")

,model.model,answer.smallest_prime,comment.smallest_prime_comment
0,gpt-4o,2,2 is the smallest prime number because it is the first number greater than 1 that is divisible by only 1 and itself.


The `Results` also include information about the question, model parameters, prompts, generated tokens and raw responses. 
To see a list of all the components:

In [10]:
results.columns

,0
0,agent.agent_index
1,agent.agent_instruction
2,agent.agent_name
3,answer.smallest_prime
4,cache_keys.smallest_prime_cache_key
5,cache_used.smallest_prime_cache_used
6,comment.smallest_prime_comment
7,generated_tokens.smallest_prime_generated_tokens
8,iteration.iteration
9,model.frequency_penalty


## Example: Conducting a survey with agents and models
In the next example we construct a more complex survey consisting of multiple questions, and design personas for AI agents to answer the survey.
Then we select specific language models to generate the answers.

We start by creating questions in different types and passing them to a `Survey`:

In [11]:
from edsl import QuestionLinearScale, QuestionFreeText

q_enjoy = QuestionLinearScale(
    question_name = "enjoy",
    question_text = "On a scale from 1 to 5, how much do you enjoy reading?",
    question_options = [1, 2, 3, 4, 5],
    option_labels = {1:"Not at all", 5:"Very much"}
)

q_favorite_place = QuestionFreeText(
    question_name = "favorite_place",
    question_text = "Describe your favorite place for reading."
)

We construct a `Survey` by passing a list of questions:

In [12]:
from edsl import Survey

survey = Survey(questions = [q_enjoy, q_favorite_place])

### Agents
An important feature of EDSL is the ability to create AI agents to answer questions.
This is done by passing dictionaries of relevant "traits" to `Agent` objects that are used by language models to generate responses.
Learn more about [designing agents](https://docs.expectedparrot.com/en/latest/agents.html).

Here we construct several simple agent personas to use with our survey:

In [13]:
from edsl import AgentList, Agent

agents = AgentList(
    Agent(traits = {"persona":p}) for p in ["artist", "mechanic", "sailor"]
)

### Language models
EDSL works with many popular large language models that we can select to use with a survey.
This makes it easy to compare responses among models in the results that are generated.

To see a current list of available models:

In [14]:
from edsl import Model

Model.available() 

,Model Name,Service Name
0,gemma-7b-it,groq
1,gemma2-9b-it,groq
2,llama-3.1-70b-versatile,groq
3,llama-3.1-8b-instant,groq
4,llama-guard-3-8b,groq
5,llama3-70b-8192,groq
6,llama3-8b-8192,groq
7,llama3-groq-70b-8192-tool-use-preview,groq
8,llama3-groq-8b-8192-tool-use-preview,groq
9,mixtral-8x7b-32768,groq


To check the default model that will be used if no models are specified for a survey (e.g., as in the first example above):

In [15]:
Model()

,key,value
0,model,gpt-4o
1,parameters:temperature,0.500000
2,parameters:max_tokens,1000
3,parameters:top_p,1
4,parameters:frequency_penalty,0
5,parameters:presence_penalty,0
6,parameters:logprobs,False
7,parameters:top_logprobs,3
8,inference_service,openai


(Note that the output may be different if the default model has changed since this page was last updated.)

Here we select some models to use with our survey:

In [16]:
from edsl import ModelList, Model

models = ModelList(
    Model(m) for m in ["gpt-4o", "gemini-1.5-flash"]
)

### Running a survey
We add agents and models to a survey using the `by` method.
Then we administer a survey the same way that we do an individual question, by calling the `run` method on it:

In [17]:
results = survey.by(agents).by(models).run()

Job UUID,4b45847d-94cb-4281-84dc-0bd3fb8db348
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/4b45847d-94cb-4281-84dc-0bd3fb8db348
Exceptions Report URL,None
Results UUID,c2e8b487-86b9-49e6-b2b2-973840233d6e
Results URL,https://www.expectedparrot.com/content/c2e8b487-86b9-49e6-b2b2-973840233d6e


We can pass an expression to `filter()` the results and list the components to `sort_by()`:

In [18]:
(
    results
    .filter("persona != 'artist'")
    .sort_by("persona", "model")
    .select("model", "persona", "enjoy", "favorite_place")
)

,model.model,agent.persona,answer.enjoy,answer.favorite_place
0,gemini-1.5-flash,mechanic,3,"My favorite place to read? Gotta be my garage, actually. Not the whole thing, mind you. There's a little alcove tucked away behind my parts bins, kinda hidden. Got a comfy old stool I salvaged from a junkyard – best darn seat I ever had. Got a good work lamp up there, too, so I can see the fine print without squinting. The air smells like oil and grease, yeah, but it's a familiar smell, comforting. Plus, it's quiet except for the occasional drip from a leaky faucet – kinda like a metronome keeping time with my reading. It's my sanctuary, you know? Away from the noise and the customers. Just me, my book, and the ghosts of a thousand engines past."
1,gpt-4o,mechanic,3,"As a mechanic, I don't get a lot of time to sit down and read, but when I do, I enjoy a spot that's a bit unconventional. My favorite place is actually in the garage, right next to my toolbox. There's something comforting about being surrounded by the hum of engines and the smell of oil and grease. I set up a small chair there, and when the shop is quiet, I can dive into a good book. The best part is when I can relate what I'm reading to what I do—especially if it's a manual or a book about classic cars."
2,gemini-1.5-flash,sailor,3,"Ahoy there! My favorite place for a good read? That'd have to be the crow's nest on a calm, clear night. The gentle rocking of the ship, the vast expanse of stars above... it's just magical. The wind whispers secrets in your ear, and the only sounds are the creak of the timbers and the distant cry of a gull. You feel so small, yet so connected to everything. Perfect for losing yourself in a yarn, whether it's a swashbuckling adventure or a quiet tale of the sea. Can't beat it."
3,gpt-4o,sailor,4,"Ah, my favorite place for reading would be aboard a ship, out on the open sea. There's nothing quite like the gentle rocking of the waves and the salty breeze to set the perfect backdrop for diving into a good book. I usually find a cozy spot on the deck, maybe near the bow where I can watch the horizon. The sound of the ocean and the occasional cry of a seagull make for a peaceful soundtrack. It's a place where time seems to slow down, and I can lose myself in the pages without a care in the world."


## Example: Adding context to questions
EDSL provides a variety of ways to add data or content to survey questions. 
These methods include:

* [Piping](https://docs.expectedparrot.com/en/latest/surveys.html#id2) answers to questions into follow-on questions
* [Adding "memory"](https://docs.expectedparrot.com/en/latest/surveys.html#question-memory) of prior questions and answers in a survey when presenting other questions to a model
* [Parameterizing questions with data](https://docs.expectedparrot.com/en/latest/scenarios.html), e.g., content from PDFs, CSVs, docs, images or other sources that you want to add to questions

### Piping question answers
Here we demonstrate how to pipe the answer to a question into the text of another question.
This is done by using a placeholder `{{ <question_name>.answer }}` in the text of the follow-on question where the answer to the prior question is to be inserted when the survey is run.
This causes the questions to be administered in the required order (survey questions are administered asynchronously by default).
Learn more about [piping question answers](https://docs.expectedparrot.com/en/latest/surveys.html#id2).

Here we insert the answer to a numerical question into the text of a follow-on yes/no question:

In [19]:
from edsl import QuestionNumerical, QuestionYesNo, Survey

q1 = QuestionNumerical(
    question_name = "random_number",
    question_text = "Pick a random number between 1 and 1,000."
)

q2 = QuestionYesNo(
    question_name = "prime",
    question_text = "Is this a prime number: {{ random_number.answer }}"
)

survey = Survey([q1, q2])

results = survey.run()

Job UUID,a9b5342c-49e4-402a-85a7-e5a7346669b5
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/a9b5342c-49e4-402a-85a7-e5a7346669b5
Exceptions Report URL,None
Results UUID,bcfb036a-ec37-4f0b-b940-cebf6bb8e637
Results URL,https://www.expectedparrot.com/content/bcfb036a-ec37-4f0b-b940-cebf6bb8e637


We can check the `user_prompt` for the `prime` question to verify that that the answer to the `random_number` question was piped into it:

In [20]:
results.select("random_number", "prime_user_prompt", "prime", "prime_comment")

,answer.random_number,prompt.prime_user_prompt,answer.prime,comment.prime_comment
0,524,"Is this a prime number: 524 No Yes Only 1 option may be selected. Please respond with just your answer. After the answer, you can put a comment explaining your response.",No,"524 is not a prime number because it is an even number greater than 2, meaning it is divisible by 2."


### Adding "memory" of questions and answers
Here we instead add a "memory" of the first question and answer to the context of the second question.
This is done by calling a memory rule and identifying the question(s) to add.
Instead of just the answer, information about the full question and answer are presented with the follow-on question text, and no placeholder is used.
Learn more about [question memory rules](https://docs.expectedparrot.com/en/latest/surveys.html#survey-rules-logic).

Here we demonstrate the `add_targeted_memory` method (we could also use `set_full_memory_mode` or other memory rules):

In [21]:
from edsl import QuestionNumerical, QuestionYesNo, Survey

q1 = QuestionNumerical(
    question_name = "random_number",
    question_text = "Pick a random number between 1 and 1,000."
)

q2 = QuestionYesNo(
    question_name = "prime",
    question_text = "Is the number you picked a prime number?"
)

survey = Survey([q1, q2]).add_targeted_memory(q2, q1)

results = survey.run()

Job UUID,0d058adb-3a79-43a2-8db6-6092672ed0f2
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/0d058adb-3a79-43a2-8db6-6092672ed0f2
Exceptions Report URL,None
Results UUID,d834cedc-1ef2-4f39-8b46-1ffd639d0928
Results URL,https://www.expectedparrot.com/content/d834cedc-1ef2-4f39-8b46-1ffd639d0928


We can again use the `user_prompt` to verify the context that was added to the follow-on question. To view the results in a long table, we can call the `table()` and `long()` methods to modify the default table view:

In [22]:
results.select("random_number", "prime_user_prompt", "prime", "prime_comment").table().long()

,row,key,value
0,0,answer.random_number,573
1,0,prompt.prime_user_prompt,"Is the number you picked a prime number? No Yes Only 1 option may be selected. Please respond with just your answer. After the answer, you can put a comment explaining your response. Before the question you are now answering, you already answered the following question(s): Question: Pick a random number between 1 and 1,000. Answer: 573"
2,0,answer.prime,No
3,0,comment.prime_comment,573 is not a prime number because it can be divided by 3 (573 ÷ 3 = 191).


*Related topic: Learn more about exploring and simulating "randomness" with AI agents and LLMs in [this notebook](https://docs.expectedparrot.com/en/latest/notebooks/random_numbers.html).*

## Scenarios
We can also add external data or content to survey questions.
This can be useful when you want to efficiently create and administer multiple versions of questions at once, e.g., for conducting data labeling tasks.
This is done by creating `Scenario` dictionaries for the data or content to be used with a survey, where the keys match `{{ placeholder }}` names used in question texts (or question options) and the values are the content to be added.
Scenarios can also be used to [add metadata to survey results](https://docs.expectedparrot.com/en/latest/notebooks/adding_metadata.html), e.g., data sources or other information that you may want to include in the results for reference but not necessarily include in question texts.

In the next example we revise the prior survey questions about reading to take a parameter for other activities that we may want to add to the questions, and create simple scenarios for some activities.
EDSL provides methods for automatically generating scenarios from a variety of data sources, including PDFs, CSVs, docs, images, tables and dicts. 
We use the `from_list` method to convert a list of activities into scenarios.

Then we demonstrate how to use scenarios to create multiple versions of our questions either (i) when constructing a survey or (ii) when running it:

* In the latter case, the `by` method is used to add scenarios to a survey of questions with placeholders at the time that it is run (the same way that agents and models are added to a survey). This adds a `scenario` column to the results with a row for each answer to each question for each scenario.
* In the former case, the `loop` method is used to create a list of versions of a question with the scenarios already added to it; when the questions are passed to a survey and it is run, the results include columns for each individual question; there is no `scenario` column and a single row for each agent's answers to all the questions.

Learn more about [using scenarios](https://docs.expectedparrot.com/en/latest/scenarios.html).

Here we create scenarios for a simple list of activities:

In [23]:
from edsl import ScenarioList, Scenario

scenarios = ScenarioList.from_list("activity", ["reading", "running", "relaxing"])

### Adding scenarios using the `by` method
Here we add the scenarios to the survey when we run it, together with any desired agents and models:

In [24]:
from edsl import QuestionLinearScale, QuestionFreeText, Survey

q_enjoy = QuestionLinearScale(
    question_name = "enjoy",
    question_text = "On a scale from 1 to 5, how much do you enjoy {{ activity }}?",
    question_options = [1, 2, 3, 4, 5],
    option_labels = {1:"Not at all", 5:"Very much"}
)

q_favorite_place = QuestionFreeText(
    question_name = "favorite_place",
    question_text = "In a brief sentence, describe your favorite place for {{ activity }}."
)

survey = Survey([q_enjoy, q_favorite_place])

In [25]:
results = survey.by(scenarios).by(agents).by(models).run()

Job UUID,53b10503-9c2b-4205-89eb-b2d1ec794bac
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/53b10503-9c2b-4205-89eb-b2d1ec794bac
Exceptions Report URL,None
Results UUID,bc33c1ce-6b8f-4f34-ba96-02df2bf9b643
Results URL,https://www.expectedparrot.com/content/bc33c1ce-6b8f-4f34-ba96-02df2bf9b643


In [26]:
(
    results
    .filter("model.model == 'gpt-4o'")
    .sort_by("activity", "persona")
    .select("activity", "persona", "enjoy", "favorite_place")
)

,scenario.activity,agent.persona,answer.enjoy,answer.favorite_place
0,reading,artist,4,"My favorite place for reading is a cozy nook by a large window, where natural light floods in, surrounded by plants and soft cushions."
1,reading,mechanic,3,"My favorite place for reading is a cozy corner in the garage, where the smell of engine oil and the hum of a nearby project create a unique and comforting atmosphere."
2,reading,sailor,4,"My favorite place for reading is on the deck of a ship, with the salty breeze and the sound of waves as my backdrop."
3,relaxing,artist,4,"My favorite place for relaxing is a quiet, sunlit studio filled with canvases, brushes, and the gentle hum of creativity in the air."
4,relaxing,mechanic,4,"My favorite place for relaxing is my garage, surrounded by the familiar scent of motor oil and the hum of engines."
5,relaxing,sailor,4,"My favorite place for relaxing is on the deck of a ship, feeling the gentle sway of the waves and the salty breeze on my face."
6,running,artist,2,"My favorite place for running is a quiet forest trail where the sunlight filters through the leaves, creating a moving canvas of light and shadow."
7,running,mechanic,1,"I don't run much, but if I did, I'd probably enjoy a trail through a forest, where the air feels fresh and the ground is a bit softer on the knees."
8,running,sailor,2,"My favorite place for running is the sandy shores of a quiet beach at dawn, with the sound of the waves crashing and the salty breeze in the air."


### Adding scenarios using the `loop` method
Here we add scenarios to questions when constructing a survey, as opposed to when running it.
When we run the survey the results will include columns for each question and no `scenario` field. 
Note that we can also optionally use the scenario key in the question names (they are otherwise incremented by default):

In [27]:
from edsl import QuestionLinearScale, QuestionFreeText

q_enjoy = QuestionLinearScale(
    question_name = "enjoy_{{ activity }}", # optional use of scenario key
    question_text = "On a scale from 1 to 5, how much do you enjoy {{ activity }}?",
    question_options = [1, 2, 3, 4, 5],
    option_labels = {1:"Not at all", 5:"Very much"}
)

q_favorite_place = QuestionFreeText(
    question_name = "favorite_place_{{ activity }}", # optional use of scenario key
    question_text = "In a brief sentence, describe your favorite place for {{ activity }}."
)

Looping the scenarios to create lists of questions:

In [28]:
enjoy_questions = q_enjoy.loop(scenarios)
enjoy_questions

[Question('linear_scale', question_name = """enjoy_reading""", question_text = """On a scale from 1 to 5, how much do you enjoy reading?""", question_options = [1, 2, 3, 4, 5], option_labels = {1: 'Not at all', 5: 'Very much'}),
 Question('linear_scale', question_name = """enjoy_running""", question_text = """On a scale from 1 to 5, how much do you enjoy running?""", question_options = [1, 2, 3, 4, 5], option_labels = {1: 'Not at all', 5: 'Very much'}),
 Question('linear_scale', question_name = """enjoy_relaxing""", question_text = """On a scale from 1 to 5, how much do you enjoy relaxing?""", question_options = [1, 2, 3, 4, 5], option_labels = {1: 'Not at all', 5: 'Very much'})]

In [29]:
favorite_place_questions = q_favorite_place.loop(scenarios)
favorite_place_questions

[Question('free_text', question_name = """favorite_place_reading""", question_text = """In a brief sentence, describe your favorite place for reading."""),
 Question('free_text', question_name = """favorite_place_running""", question_text = """In a brief sentence, describe your favorite place for running."""),
 Question('free_text', question_name = """favorite_place_relaxing""", question_text = """In a brief sentence, describe your favorite place for relaxing.""")]

Combining the questions in a survey:

In [30]:
survey = Survey(questions = enjoy_questions + favorite_place_questions)

In [31]:
results = survey.by(agents).by(models).run()

Job UUID,5688ce6e-01d4-4032-93c1-4945ac99ed2a
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/5688ce6e-01d4-4032-93c1-4945ac99ed2a
Exceptions Report URL,None
Results UUID,d6ea292a-4bae-4e9e-b73e-a87012807bc0
Results URL,https://www.expectedparrot.com/content/d6ea292a-4bae-4e9e-b73e-a87012807bc0


We can see that there are additional question fields and no "scenario" field:

In [32]:
results.columns 

,0
0,agent.agent_index
1,agent.agent_instruction
2,agent.agent_name
3,agent.persona
4,answer.enjoy_reading
5,answer.enjoy_relaxing
6,answer.enjoy_running
7,answer.favorite_place_reading
8,answer.favorite_place_relaxing
9,answer.favorite_place_running


In [33]:
(
    results
    .filter("model.model == 'gpt-4o'")
    .sort_by("persona")
    .select("persona", "enjoy_reading", "enjoy_running", "enjoy_relaxing", "favorite_place_reading", "favorite_place_running", "favorite_place_relaxing")
)

,agent.persona,answer.enjoy_reading,answer.enjoy_running,answer.enjoy_relaxing,answer.favorite_place_reading,answer.favorite_place_running,answer.favorite_place_relaxing
0,artist,4,2,4,"My favorite place for reading is a cozy nook by a large window, where natural light spills over my art books and the world outside becomes a living painting.",My favorite place for running is a serene forest trail where the earthy scent of pine and the dappled sunlight create an inspiring natural canvas.,"My favorite place for relaxing is a cozy corner of my studio, surrounded by vibrant canvases and the gentle hum of creativity."
1,mechanic,3,2,3,"My favorite place for reading is a cozy corner in my garage, surrounded by the smell of motor oil and the hum of tools, where I can escape into a good book after a long day of fixing engines.","I really enjoy running along the scenic trails by the river, where the sound of water and the rustling leaves create a peaceful atmosphere.","My favorite place for relaxing is my garage, where I can unwind tinkering with engines and tools, surrounded by the familiar scent of oil and metal."
2,sailor,4,2,3,"My favorite place for reading is the deck of a ship, with the gentle sway of the sea and the salty breeze filling the air.",My favorite place for running is along the coastal paths where the sea breeze and the sound of waves keep me company.,"There's nothing quite like the gentle sway of a hammock aboard a sailboat, anchored in a secluded cove, with the sound of waves lapping against the hull."


## Exploring `Results`
EDSL comes with [built-in methods for analyzing and visualizing survey results](https://docs.expectedparrot.com/en/latest/language_models.html). 
For example, you can call the `to_pandas` method to convert results into a dataframe:

In [34]:
df = results.to_pandas(remove_prefix=True)
df

,enjoy_running,favorite_place_reading,favorite_place_relaxing,enjoy_relaxing,enjoy_reading,favorite_place_running,scenario_index,agent_name,agent_index,agent_instruction,...,favorite_place_reading_cache_used,enjoy_relaxing_cache_used,favorite_place_relaxing_cache_used,enjoy_running_cache_used,favorite_place_running_cache_key,favorite_place_relaxing_cache_key,enjoy_reading_cache_key,enjoy_running_cache_key,favorite_place_reading_cache_key,enjoy_relaxing_cache_key
0,2,My favorite place for reading is a cozy nook b...,My favorite place for relaxing is a cozy corne...,4,4,My favorite place for running is a serene fore...,0,Agent_27,0,You are answering questions as if you were a h...,...,False,False,False,False,3eb6f64c9cec7a0c3afefa9ec6fb3850,66f0a7975cfaefa813fc305cff6cd5ac,aebeeb0f2fb668a3e6664a2bfd0a72e0,9e6d02b64897f2e4331fda85fd9b67ea,81d429e0e1a56da26d6e69af1892b035,58af602684040393e974b76e2520c301
1,1,Honestly? Anywhere the light's good and I've ...,My favorite place to relax is nestled in my st...,5,3,Anywhere with a good view – preferably overloo...,0,Agent_28,0,You are answering questions as if you were a h...,...,False,False,False,False,5562aa24b919d77287b8bd643ede61b9,299ca52d321d41e9e39a2ed6d491b94a,3ac7e9c78caef3ede197c8fd8e0df14e,d73f06ab735eb86ebf28b2bf80ef133d,e36d61037f93bc5f73ecc1645b579a86,a185f7d691a33d906bc69a3b94143ab6
2,2,My favorite place for reading is a cozy corner...,"My favorite place for relaxing is my garage, w...",3,3,I really enjoy running along the scenic trails...,0,Agent_29,1,You are answering questions as if you were a h...,...,False,False,False,False,9b03af8fd0c9fa0fe555f38708be0694,f4d25e2577d27547d68136279c88764d,2c313b0030f71fc398cfe7e991d3adaa,5eeb5dbbcd29aeb5c2b0b2b2e0ab6155,8e87911cd27901026e712fb5436d0e37,6a4e831435ccd63c37d6bec0bf30aa2e
3,1,My favorite place to read? Gotta be my garage...,"My garage, with a cold beer and a good engine ...",3,3,"Anywhere with a good, solid, well-maintained a...",0,Agent_30,1,You are answering questions as if you were a h...,...,False,False,False,False,8eec64a458062c528e75a572cc365d39,8db7c7286e707f8e9810b2d014731c23,83451014aac4502558c2d9837b681449,3165de629a9437fd777474a7a44e521b,2fd1f64e7dd01a32808d23d072916ec1,b9716769febe43b34813ec9b18ee6b5a
4,2,My favorite place for reading is the deck of a...,There's nothing quite like the gentle sway of ...,3,4,My favorite place for running is along the coa...,0,Agent_31,2,You are answering questions as if you were a h...,...,False,False,False,False,e47082471a3fbd466b67dd408d4f3955,001e333ad5b2b9e502406f72595130d8,fe9ee22444155d896a498d300d2ef494,adb37dc4e0d6f7c9c953edf66c729e10,de03007625800b53fc871284064b1b13,32251e89dd8481dbe21c08aebce941d3
5,1,"The crow's nest, of course; the wind in my hai...","A quiet cove, somewhere the waves whisper secr...",4,3,"The beach at dawn, the sand cool beneath my fe...",0,Agent_32,2,You are answering questions as if you were a h...,...,False,False,False,False,6e8a7b235abdf44ebce79e3700d7cd5e,cfa586c561784b49ae6284bc511c6ecb,478e781444116ecbb2785a3c470d82f0,8b46c42710538a9f0885028b796edeb3,29e10fc028d84a010ec78d0579d8cac1,49e39e717d88d6a16d20fa4dc3af848b


The `Results` object also supports SQL-like queries with the the `sql` method:

In [35]:
results.sql("""
select model, persona, enjoy_reading, favorite_place_reading
from self
order by 1,2,3
""")

,model,persona,enjoy_reading,favorite_place_reading
0,gemini-1.5-flash,artist,3,Honestly? Anywhere the light's good and I've got a steaming mug of something delicious nearby.
1,gemini-1.5-flash,mechanic,3,"My favorite place to read? Gotta be my garage, surrounded by the comforting smell of oil and grease. Nothing beats it."
2,gemini-1.5-flash,sailor,3,"The crow's nest, of course; the wind in my hair, the endless sea spread before me... perfect."
3,gpt-4o,artist,4,"My favorite place for reading is a cozy nook by a large window, where natural light spills over my art books and the world outside becomes a living painting."
4,gpt-4o,mechanic,3,"My favorite place for reading is a cozy corner in my garage, surrounded by the smell of motor oil and the hum of tools, where I can escape into a good book after a long day of fixing engines."
5,gpt-4o,sailor,4,"My favorite place for reading is the deck of a ship, with the gentle sway of the sea and the salty breeze filling the air."


## Posting to the Coop
The [Coop](https://www.expectedparrot.com/content/explore) is a platform for creating, storing and sharing LLM-based research.
It is fully integrated with EDSL and accessible from your workspace or Coop account page.
Learn more about [creating an account](https://www.expectedparrot.com/login) and [using the Coop](https://docs.expectedparrot.com/en/latest/coop.html).

We can post any EDSL object to the Coop by call the `push` method on it, optionally passing a `description` and `visibility` status.

For example, the results above are already posted to Coop because they were generated using remote inference (see links). 
Here we show how to post them manually:

In [36]:
results.push(description = "Starter tutorial sample survey results", visibility="public")

{'description': 'Starter tutorial sample survey results',
 'object_type': 'results',
 'url': 'https://www.expectedparrot.com/content/f1501307-f29e-4a56-9e6e-de70e9ca32bb',
 'uuid': 'f1501307-f29e-4a56-9e6e-de70e9ca32bb',
 'version': '0.1.43.dev1',
 'visibility': 'public'}

We can also post this notebook:

In [37]:
from edsl import Notebook

notebook = Notebook(path="starter_tutorial.ipynb")

info = notebook.push(description="Starter Tutorial", visibility="public")
info

{'description': 'Starter Tutorial',
 'object_type': 'notebook',
 'url': 'https://www.expectedparrot.com/content/179b3a78-2505-4568-acd9-c09d18953288',
 'uuid': '179b3a78-2505-4568-acd9-c09d18953288',
 'version': '0.1.43.dev1',
 'visibility': 'public'}

To update an object:

In [38]:
from edsl import Notebook

notebook = Notebook(path="starter_tutorial.ipynb") # resave

notebook.patch(uuid = "179b3a78-2505-4568-acd9-c09d18953288", value = notebook)

{'status': 'success'}